In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import psycopg2

In [2]:
#Import from AWS

In [3]:
conn = psycopg2.connect(database="HeartDiseaseProject_db",
                        host="finalhdproject.cxmlmtuwfiqd.us-east-2.rds.amazonaws.com",
                        user="group4",
                        password="randomforest4",
                        port="5432")

In [4]:
cur = conn.cursor()

In [5]:
cur.execute('SELECT * FROM Morbidity_df')

In [6]:
Morbidity_df = cur.fetchall()
Morbidity_df = pd.DataFrame(Morbidity_df)
Morbidity_df

,0,1,2,3,4,5,6,7
0,0,False,False,False,Yes,True,False,True
1,1,False,True,False,No,False,False,False
2,2,False,False,False,Yes,True,False,False
3,3,False,False,False,No,False,False,True
4,4,False,False,True,No,False,False,False
...,...,...,...,...,...,...,...,...
319790,319790,True,False,True,Yes,True,False,False
319791,319791,False,False,False,No,True,False,False
319792,319792,False,False,False,No,False,False,False
319793,319793,False,False,False,No,False,False,False


In [7]:
cur.execute('SELECT * FROM Demographics_df')

In [8]:
Demographics_df = cur.fetchall()
Demographics_df = pd.DataFrame(Demographics_df)
Demographics_df

,0,1,2,3
0,0,Female,55-59,White
1,1,Female,80 or older,White
2,2,Male,65-69,White
3,3,Female,75-79,White
4,4,Female,40-44,White
...,...,...,...,...
319790,319790,Male,60-64,Hispanic
319791,319791,Male,35-39,Hispanic
319792,319792,Female,45-49,Hispanic
319793,319793,Female,25-29,Hispanic


In [9]:
cur.execute('SELECT * FROM HealthMetrics_df')

In [10]:
HealthMetrics = cur.fetchall()
HealthMetrics = pd.DataFrame(HealthMetrics)
HealthMetrics

,0,1,2,3,4,5,6,7,8
0,0,16.60,True,False,3,30,True,Very good,5
1,1,20.34,False,False,0,0,True,Very good,7
2,2,26.58,True,False,20,30,True,Fair,8
3,3,24.21,False,False,0,0,False,Good,6
4,4,23.71,False,False,28,0,True,Very good,8
...,...,...,...,...,...,...,...,...,...
319790,319790,27.41,True,False,7,0,False,Fair,6
319791,319791,29.84,True,False,0,0,True,Very good,5
319792,319792,24.24,False,False,0,0,True,Good,6
319793,319793,32.81,False,False,0,0,False,Good,12


In [11]:
Morbidity_df.columns = ['RespondentID', 'HeartDisease', 'Stroke', 'DifficultyWalking', 'Diabetes', 'Asthma', 'KidneyDisease', 'SkinCancer']

In [12]:
Demographics_df.columns = ['RespondentID', 'Sex', 'AgeRange', 'Race']

In [13]:
HealthMetrics.columns=['RespondentID', 'BMI', 'Smoking', 'AlcoholDrinking', 'HealthyPhysicalDays', 'HealthyMentalDays', 'PhysicalActivity', 'GeneralHealth', 'SleepHours']

In [14]:
Heart_df = Morbidity_df.merge(Demographics_df, on='RespondentID')
Heart_df = Heart_df.merge(HealthMetrics, on='RespondentID')
Heart_df

,RespondentID,HeartDisease,Stroke,DifficultyWalking,Diabetes,Asthma,KidneyDisease,SkinCancer,Sex,AgeRange,Race,BMI,Smoking,AlcoholDrinking,HealthyPhysicalDays,HealthyMentalDays,PhysicalActivity,GeneralHealth,SleepHours
0,0,False,False,False,Yes,True,False,True,Female,55-59,White,16.60,True,False,3,30,True,Very good,5
1,1,False,True,False,No,False,False,False,Female,80 or older,White,20.34,False,False,0,0,True,Very good,7
2,2,False,False,False,Yes,True,False,False,Male,65-69,White,26.58,True,False,20,30,True,Fair,8
3,3,False,False,False,No,False,False,True,Female,75-79,White,24.21,False,False,0,0,False,Good,6
4,4,False,False,True,No,False,False,False,Female,40-44,White,23.71,False,False,28,0,True,Very good,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,319790,True,False,True,Yes,True,False,False,Male,60-64,Hispanic,27.41,True,False,7,0,False,Fair,6
319791,319791,False,False,False,No,True,False,False,Male,35-39,Hispanic,29.84,True,False,0,0,True,Very good,5
319792,319792,False,False,False,No,False,False,False,Female,45-49,Hispanic,24.24,False,False,0,0,True,Good,6
319793,319793,False,False,False,No,False,False,False,Female,25-29,Hispanic,32.81,False,False,0,0,False,Good,12


In [15]:
Heart_df = Heart_df.drop(columns=['RespondentID'])
Heart_df

,HeartDisease,Stroke,DifficultyWalking,Diabetes,Asthma,KidneyDisease,SkinCancer,Sex,AgeRange,Race,BMI,Smoking,AlcoholDrinking,HealthyPhysicalDays,HealthyMentalDays,PhysicalActivity,GeneralHealth,SleepHours
0,False,False,False,Yes,True,False,True,Female,55-59,White,16.60,True,False,3,30,True,Very good,5
1,False,True,False,No,False,False,False,Female,80 or older,White,20.34,False,False,0,0,True,Very good,7
2,False,False,False,Yes,True,False,False,Male,65-69,White,26.58,True,False,20,30,True,Fair,8
3,False,False,False,No,False,False,True,Female,75-79,White,24.21,False,False,0,0,False,Good,6
4,False,False,True,No,False,False,False,Female,40-44,White,23.71,False,False,28,0,True,Very good,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,True,False,True,Yes,True,False,False,Male,60-64,Hispanic,27.41,True,False,7,0,False,Fair,6
319791,False,False,False,No,True,False,False,Male,35-39,Hispanic,29.84,True,False,0,0,True,Very good,5
319792,False,False,False,No,False,False,False,Female,45-49,Hispanic,24.24,False,False,0,0,True,Good,6
319793,False,False,False,No,False,False,False,Female,25-29,Hispanic,32.81,False,False,0,0,False,Good,12


In [16]:
#Encode catagorical data
Heart_encoded = pd.get_dummies(Heart_df, columns=['HeartDisease','Sex','AgeRange','Smoking', 'Stroke','Diabetes','PhysicalActivity', 'AlcoholDrinking','DifficultyWalking','Race','GeneralHealth', 'Asthma','KidneyDisease','SkinCancer',])
Heart_encoded.head()

,BMI,HealthyPhysicalDays,HealthyMentalDays,SleepHours,HeartDisease_False,HeartDisease_True,Sex_Female,Sex_Male,AgeRange_18-24,AgeRange_25-29,...,GeneralHealth_Fair,GeneralHealth_Good,GeneralHealth_Poor,GeneralHealth_Very good,Asthma_False,Asthma_True,KidneyDisease_False,KidneyDisease_True,SkinCancer_False,SkinCancer_True
0,16.60,3,30,5,1,0,1,0,0,0,...,0,0,0,1,0,1,1,0,0,1
1,20.34,0,0,7,1,0,1,0,0,0,...,0,0,0,1,1,0,1,0,1,0
2,26.58,20,30,8,1,0,0,1,0,0,...,1,0,0,0,0,1,1,0,1,0
3,24.21,0,0,6,1,0,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
4,23.71,28,0,8,1,0,1,0,0,0,...,0,0,0,1,1,0,1,0,1,0


In [17]:
Heart_encoded.describe()

,HealthyPhysicalDays,HealthyMentalDays,SleepHours,HeartDisease_False,HeartDisease_True,Sex_Female,Sex_Male,AgeRange_18-24,AgeRange_25-29,AgeRange_30-34,...,GeneralHealth_Fair,GeneralHealth_Good,GeneralHealth_Poor,GeneralHealth_Very good,Asthma_False,Asthma_True,KidneyDisease_False,KidneyDisease_True,SkinCancer_False,SkinCancer_True
count,319795.00000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,...,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000,319795.000000
mean,3.37171,3.898366,7.097075,0.914405,0.085595,0.524727,0.475273,0.065867,0.053018,0.058641,...,0.108435,0.291215,0.035301,0.356034,0.865939,0.134061,0.963167,0.036833,0.906756,0.093244
std,7.95085,7.955235,1.436007,0.279766,0.279766,0.499389,0.499389,0.248050,0.224070,0.234951,...,0.310930,0.454323,0.184539,0.478826,0.340718,0.340718,0.188352,0.188352,0.290775,0.290775
min,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,0.00000,0.000000,7.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,2.00000,3.000000,8.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,30.00000,30.000000,24.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
#Subset data to use for balancing 

In [19]:
positive = Heart_encoded.loc[Heart_encoded['HeartDisease_True']==1]
positive = positive.sample(300)

In [20]:
negative = Heart_encoded.loc[Heart_encoded['HeartDisease_True']==0]
negative = negative.sample(300)

In [21]:
Asthma = Heart_encoded.loc[Heart_encoded['Asthma_True']==1]
Asthma = Asthma.sample(300)

In [22]:
Kidney = Heart_encoded.loc[Heart_encoded['KidneyDisease_True']==1]
Kidney = Kidney.sample(300)

In [23]:
Stroke = Heart_encoded.loc[Heart_encoded['Stroke_True']==1]
Stroke = Stroke.sample(600)

In [24]:
Skincan = Heart_encoded.loc[Heart_encoded['SkinCancer_True']==1]
Skincan = Skincan.sample(300)

In [25]:
Asian = Heart_encoded.loc[Heart_encoded['Race_Asian']==1]
Asian = Asian.sample(200)

In [26]:
Black = Heart_encoded.loc[Heart_encoded['Race_Black']==1]
Black = Black.sample(150)

In [27]:
Hispanic = Heart_encoded.loc[Heart_encoded['Race_Hispanic']==1]
Hispanic = Hispanic.sample(150)

In [28]:
Native = Heart_encoded.loc[Heart_encoded['Race_American Indian/Alaskan Native']==1]
Native = Native.sample(200)

In [29]:
Other = Heart_encoded.loc[Heart_encoded['Race_Other']==1]
Other = Other.sample(300)

In [30]:
#Balance Heart Disease 
Balanced_heart = positive.append(negative)
Balanced_heart = Balanced_heart.append(Asthma)
Balanced_heart = Balanced_heart.append(Stroke)
Balanced_heart = Balanced_heart.append(Kidney)
Balanced_heart = Balanced_heart.append(Black)
Balanced_heart = Balanced_heart.append(Asian)
Balanced_heart = Balanced_heart.append(Hispanic)
Balanced_heart = Balanced_heart.append(Native)


/var/folders/2b/k93wkrsj2ms_nw2rs0_s6pk40000gn/T/ipykernel_44413/356771239.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Balanced_heart = positive.append(negative)
/var/folders/2b/k93wkrsj2ms_nw2rs0_s6pk40000gn/T/ipykernel_44413/356771239.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Balanced_heart = Balanced_heart.append(Asthma)
/var/folders/2b/k93wkrsj2ms_nw2rs0_s6pk40000gn/T/ipykernel_44413/356771239.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Balanced_heart = Balanced_heart.append(Stroke)
/var/folders/2b/k93wkrsj2ms_nw2rs0_s6pk40000gn/T/ipykernel_44413/356771239.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [31]:
Final_heart = Balanced_heart.copy()

In [32]:
Final_heart = Final_heart.drop_duplicates()

In [33]:
Final_heart = Final_heart.reset_index(drop=True)

In [34]:
#Final_heart = Final_heart.drop(columns='Unnamed: 0')

In [35]:
Final_heart.to_excel('/Users/christopheroakes/Documents/Coding Bootcamp/Final Project/Heart_bal_clean.xlsx')

In [36]:
#Run Random Forest Model on Final Data

In [37]:
# Define the features set.
X = Final_heart.copy()
X = X.drop(["HeartDisease_True"], axis=1)
X = X.drop(["HeartDisease_False"], axis=1)
#X = X.drop(["Unnamed: 0"], axis=1)
X.head()

,BMI,HealthyPhysicalDays,HealthyMentalDays,SleepHours,Sex_Female,Sex_Male,AgeRange_18-24,AgeRange_25-29,AgeRange_30-34,AgeRange_35-39,...,GeneralHealth_Fair,GeneralHealth_Good,GeneralHealth_Poor,GeneralHealth_Very good,Asthma_False,Asthma_True,KidneyDisease_False,KidneyDisease_True,SkinCancer_False,SkinCancer_True
0,25.70,0,0,8,1,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
1,27.46,0,0,8,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
2,26.19,0,0,9,0,1,0,0,0,0,...,0,0,0,1,1,0,1,0,0,1
3,32.22,30,30,8,1,0,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
4,23.40,0,5,8,0,1,0,0,0,0,...,0,0,0,1,1,0,1,0,0,1


In [38]:
# Define the target set.
y = Final_heart["HeartDisease_True"].ravel()
y[:5]

array([1, 1, 1, 1, 1], dtype=uint8)

In [39]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [40]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

1863
622
1863
622


In [41]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=42) 

In [43]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [44]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [45]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,410,30
Actual 1,125,57


In [46]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [47]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,410,30
Actual 1,125,57


Accuracy Score : 0.7508038585209004
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.93      0.84       440
           1       0.66      0.31      0.42       182

    accuracy                           0.75       622
   macro avg       0.71      0.62      0.63       622
weighted avg       0.73      0.75      0.72       622



In [48]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.13190156, 0.06234329, 0.04922243, 0.07347451, 0.02026275,
       0.0203773 , 0.00146861, 0.00325316, 0.00371224, 0.00486177,
       0.0040745 , 0.00715452, 0.00918736, 0.01541013, 0.01831895,
       0.01879744, 0.01703719, 0.02297181, 0.02372078, 0.0189755 ,
       0.0201458 , 0.02265573, 0.0232004 , 0.01930111, 0.00554713,
       0.01867146, 0.00126533, 0.0169825 , 0.01700583, 0.00558473,
       0.0056671 , 0.02128449, 0.02501113, 0.00960978, 0.00694088,
       0.0116114 , 0.00876788, 0.00965725, 0.03536505, 0.01177392,
       0.01996169, 0.02045975, 0.02068711, 0.01701228, 0.01748594,
       0.01704259, 0.01727985, 0.01860958, 0.01511115, 0.01377537])

In [49]:
# We can sort the features by their importance.
feature_imp = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [50]:
feature_imp = pd.DataFrame(feature_imp)
feature_imp

,0,1
0,0.131902,BMI
1,0.073475,SleepHours
2,0.062343,HealthyPhysicalDays
3,0.049222,HealthyMentalDays
4,0.035365,Race_White
5,0.025011,DifficultyWalking_True
6,0.023721,AgeRange_80 or older
7,0.023200,Stroke_True
8,0.022972,AgeRange_75-79
9,0.022656,Stroke_False
